# READ THIS BEFORE RUNNING THIS NOTEBOOK.
## Create a Postgres DB Named 'jeoparty'
1. Use pgAdmin to create a postgres database named 'jeoparty'.
2. Run the sql script found in 'jeo_tables.sql' in the 'jeoparty' query tool.
 - This file is found in the same folder as this notebook. 
 - This will create tables to be populated by data pulled from various sources.
 - No capital letters will be used in the naming of the tables, table columns or dataframe columns
 - postgres has issues with capital letters
3. Add your pgAdmin username and password to the 'jeoparty_passwords.py' file before running any cells in this workbook.
 - 'jeoparty_passwords.py' is also found in the same folder as this notebook.
 - These values have been defaulted to 'postgres' for both variables in the 'jeoparty_passwords.py' file in the repository

# Pulling Contestants and Questions

In [1]:
#Dependencies
import pandas as pd
import requests
import json
from sqlalchemy import create_engine
from jeoparty_passwords import jeo_username
from jeoparty_passwords import jeo_password

In [2]:
# JSON file containing information for the 1000 winningest Jeopardy! Contestants pulled from
# https://cluebase.readthedocs.io/en/latest/# , a Jeopardy! API with excellent documenation.
contestant_file = "../Resources/contestants1000.json"
# JSON file containing information for Jeopardy! questions from jService.
question_file = "../static/js/cleanedJeoQs.json"

In [3]:
#Read 'player' JSON into the dataframe using pandas.read_json. Only the 'data' objects will be read in.
player_df = pd.read_json(contestant_file, orient='values')['data']
player_df.head(3)

0    {'id': 208, 'name': 'Ken Jennings', 'notes': '...
1    {'id': 75, 'name': 'James Holzhauer', 'notes':...
2    {'id': 204, 'name': 'David Madden', 'notes': '...
Name: data, dtype: object

In [4]:
#Read 'question' JSON into the dataframe using pandas.read_json.
question_df = pd.read_json(question_file, orient='values')

In [5]:
#pandas.json_normalize is used to 'flatten' the json in the 'player_df' dataframe.
#'contestant_id' will be used as the index.
player_df = pd.json_normalize(player_df).set_index('id') 
player_df.index.names = ['contestant_id']

In [6]:
#Players Dataframe.
player_df.head(3)

,name,notes,games_played,total_winnings
contestant_id,,,,
208,Ken Jennings,"a software engineer from Salt Lake City, Utah",94,2522700
75,James Holzhauer,"a professional sports gambler from Las Vegas, ...",33,2464216
204,David Madden,"a student originally from Ridgewood, New Jersey",29,432400


In [7]:
#Read 'question' JSON into the dataframe using pandas.read_json.
question_df = pd.read_json(question_file, orient='values')
question_df.index.rename("question_id", inplace = True)
question_df.head(3)

,show_number,air_date,round,category,value,question,answer
question_id,,,,,,,
0,4576,6/28/2004,Double Jeopardy!,BEFORE & AFTER,400,1980 scarefest in which mom & daughter switch ...,Freaky Friday the 13th
1,4576,6/28/2004,Double Jeopardy!,BEFORE & AFTER,800,Leif Ericson's dad who was a huge star with lo...,Erik the Red Giant
2,4576,6/28/2004,Double Jeopardy!,BEFORE & AFTER,1200,Fictional girl sleuth who's the granddaughter ...,Nancy Drew Barrymore


# Pulling Categories

In [8]:
#Pulling Categories
url = "http://cluebase.lukelav.in/categories"
response = requests.get(url)

In [9]:
# Storing the JSON response within a variable
data = response.json()
# Use json.dumps to print the json
print(json.dumps(data, indent=4, sort_keys=True))

{
    "data": [
        {
            "category": "SCIENCE",
            "clue_count": 911
        },
        {
            "category": "LITERATURE",
            "clue_count": 812
        },
        {
            "category": "AMERICAN HISTORY",
            "clue_count": 778
        },
        {
            "category": "POTPOURRI",
            "clue_count": 744
        },
        {
            "category": "HISTORY",
            "clue_count": 723
        },
        {
            "category": "SPORTS",
            "clue_count": 693
        },
        {
            "category": "BEFORE & AFTER",
            "clue_count": 677
        },
        {
            "category": "WORLD HISTORY",
            "clue_count": 649
        },
        {
            "category": "WORD ORIGINS",
            "clue_count": 631
        },
        {
            "category": "WORLD GEOGRAPHY",
            "clue_count": 612
        },
        {
            "category": "TRANSPORTATION",
            "clue_count": 569
   

In [10]:
#Parsing to make sure I'm going into the json file correctly
data['data'][0]

{'category': 'SCIENCE', 'clue_count': 911}

In [11]:
#Creating a dataframe out of the json file
df = pd.DataFrame(data['data'])
df.head(10)

,category,clue_count
0,SCIENCE,911
1,LITERATURE,812
2,AMERICAN HISTORY,778
3,POTPOURRI,744
4,HISTORY,723
5,SPORTS,693
6,BEFORE & AFTER,677
7,WORLD HISTORY,649
8,WORD ORIGINS,631
9,WORLD GEOGRAPHY,612


# Connection to Database and Populating Tables

In [12]:
# Create Connection to 'jeoparty' postgres database
# This connection string will use information from the 'jeoparty_passwords.py' file.
connection_string = f"{jeo_username}:{jeo_password}@localhost:5432/jeoparty"
engine = create_engine(f'postgresql://{connection_string}')

In [13]:
# Confirm table name 'contestants' exists.
engine.table_names()

/var/folders/_v/q5hgp4y91q79zfpg0c6yckfm0000gn/T/ipykernel_2411/1246274019.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['contestants', 'categories', 'questions']

In [14]:
#Append data in 'player_df' dataframe to 'contestants' table
player_df.to_sql(name='contestants', con=engine, if_exists='append', index=True)
#Append data in 'question_df' dataframe to 'questions' table
question_df.to_sql(name='questions', con=engine, if_exists='append', index=True)
#Append data in 'df' dataframe to 'categories' table
df.to_sql(name='categories', con=engine, if_exists='append', index=True)

50